# Report : Project of Basics of Mobile Robotics 

Group members :

- Cirillo Thomas
- Muller Nathan
- Sahin Ali Fuat
- Vinet Pierre

# TODO
- sous parties table of contents
- link table of content
- import libraries
INTRODUCTION
- image environment 
- we didn't use any libraries for the algorithms (other than numpy etc... )
- description of aruc markers (maybe in the image processing part)
I IMAGE PROCESSING
- implement images processing (image of the obstacles from the camera, detection markers, image of the processed environment )
II PATH FINDING
- more images? 
III GLOBAL NAVIGATION
- image copmaring a-star path to real pd implementation
IV LOCAL OBSTACLE
- plot of the thymio coming back to the path
V KALMAN FILTER
- plot image with camera and without
CONCLUSION
- conclusion sur les solutions apporter et limitations



## Table of Contents

- [1. Introduction](#introduction)
- [2. Image Processing](#image-processing)
- [3. Path Finding](#path-finding)
- [4. Global Navigation](#global-navigation)
- [5. Local Obstacle Avoidance](#local-obstacle-avoidance)
- [6. Kalman Filter](#kalman-filter)
- [7. Conclusion](#conclusion)


## 1. Introduction

### 1.1 The Environment 


<div style="text-align: center;">
  <img src="images\environment-description.png"alt="environment" id="img_1.1" width="1000" >
  <p style="text-align: center;">Image 1.1: Environment</p>
</div>

Our environment consist of a white surface of 1 by 1 meters, with Aruco markers on the different corners to be able to identify them using the webcam given to us. 

We first decided to use a white surface with a black grid but then decided to go with a fully white surface instead because we use a binary threshold to detect our obstacles which would then interpret the grid as an obstacle. To correct this we would had needed to specifically extract the grid to create a mask out of it and substract it to our image in order to not detect it as an obstacle but we would not need this if we used a simple white surface. We also decided that the obstacle were not just squared one and therefor the use of a grid made less sense. 

The start and the goal are represented as Aruco markers as well. The obstacle used for the global navigation are black objects because as said before we use a binary threshold to detect them and blakc works the best for this situation, and the one used for the local avoidance are white objects because since they have the same color as the surface they are not detected with our image processing program.


### 1.2 Object-Oriented Programming

We chose an object-oriented approach because it offers a structured and intuitive way to encapsulate data and functions into objects, providing a clear and organized framework for our project. This choice allows us to efficiently manage relationships between various components, and easily modify or add new functionalities. 

### 1.3 Code Structure

The code is structured into the following files :

- Image Processing (`ImageProcessing.py`)
- Kalman Filter (`KalmanFilter.py`)
- PathFinding (`PathFinding.py`)
- Global Navigation (`Navigation.py`)
- Local Avoidance (`ThymioFunctions.py`)
- Connection To Thymio (`Thymio.py`) 
- The Main Code (`main.py`)

## 2. Image Processing

The goal of this section is to detect 4 aruco markers present at the 4 corners of the environment using the webcam given to use for the project. The reason we chose aruco markers is that we wanted markers tht the webcam could detect robustly. We tried using QR codes but we found out afterward that Aruco markers were more easily and consistently detected by the webcam than QR codes. Those were more sensible to light differences causing them to be less effectively detected than Aruco markers who seem more robust against those changes. Once the 4 corners are detected and the image has been correctly warped, it detects and updates the position of the robots and of the goal, thresholding the image to find the global obstacles and creating a grid for the pathinding algorithm to use.

Using the library OpenCV, we connect to the webcam and keep on taking frames to analyze while the camera is connected

```python
#Import the OpenCV library to use the different modules
import cv2

#Connect to the webcam 
cap = cv2.VideoCapture(0)

#As long as the webcam is connected, the frames are store in the image variable
while cap.isOpened():
		ret, image = cap.read()
```

Once the camera is connected, it will warp the image giving by the webcam to make the image fit the screen. To do so, it checks the transform_start variable : 
<ul>
   <li> if set to <b>True</b>, it calls the functions detecting Aruco markers which return the different coordinates of the different markers
   
```python
    if transform_start:
                (coords, image) = aruco_read(image, transform=True, start=True)
```
</ul>

This aruco_read function (given below) takes as argument the image that will be processed as well as two boolean variables (transform and start). 
It checks for Aruco markers on the image. An if branchement make sure that if all the wanted markers are not visible, it returns an empty array instead of coordinate as well as the original image.
Finally, if all the different markers are detected, it put their different coordinate in a list, draw 4 lines around it to make it more visible on the image and draw a red dot in its center. After it has gone through all the markers, it return the list containing the coordinates and the image with the drawn line and dots.

```python
def aruco_read(image, transform, start):
	ARUCO_DICT = {
		"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
		"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
		"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
		"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
		"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
		"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
		"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
		"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
		"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
		"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
		"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
		"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
		"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
		"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
		"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
		"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
		"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
		"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
		"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
		"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
		"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
	}

	aruco_dict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT["DICT_4X4_50"])
	aruco_params = cv2.aruco.DetectorParameters()
	aruco_det = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

    #Detect if markers are recognized and stock the different variables in corners, ids and rejected
	(corners, ids, rejected) = aruco_det.detectMarkers(image)
    
    #Check if the ids detected from the markers (if any) are the correct one
	if ids is None:
		return None, image
	elif transform and 0 in ids and 1 in ids and 2 in ids and 3 in ids:
		pass
	elif not transform and start and 4 in ids and 5 in ids:
		pass
	elif not transform and not start and (4 in ids or 5 in ids):
		pass
	else:
		return None, image
    
    #If all the correct markers are detected, put all their coordinates in a list and draw their contour and center
	coord_list = []
	ids = ids.flatten()
    
    #Goes through all the coordinates and ids of the detected markers
	for (marker_corners, marker_ids) in zip(corners, ids):
		corners = marker_corners.reshape((4,2))
		(topLeft, topRight, bottomRight, bottomLeft) = corners
        
        #Put all the coordinates and ids in a list
		coord_list.append({'ID': marker_ids, 'POS': np.squeeze(corners)})			
	
        #Draw the contour of the marker
		topRight = (int(topRight[0]), int(topRight[1]))
		bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
		bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
		topLeft = (int(topLeft[0]), int(topLeft[1]))
		cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
		cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
		cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
		cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
		
        #Draw the center of it
		cX = int((topLeft[0] + bottomRight[0]) / 2.0)
		cY = int((topLeft[1] + bottomRight[1]) / 2.0)
		cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
	return coord_list, image
```

Once this function has return the different coordinates and image to the main, it checks if if the coordinates vector is empty or not and if ny coordinates are given, it switch transform_start to <b>False</b>:

```python
if coords is not None:
    transform_start = False
continue         
```

<ul>
    <li>Finally, if the tranform_start variable is set to <b>False</b>, it calls the function change_perspective whoe goal is to warp the desired part of the image to make it fit the screen
</ul>

```python
else:
	image = change_perpective(image, coords, size)
```

This function (given below) takes as arguments the image we want to warp, the different coordinates and ids of the Aruco markers we detected previously and the size of the desired output. Going through all the markers detected, an if branchement checks if the procesed marker is on of the corners and put it as entry coordinates for the warp. After this, it put all the given entry coordinates in a corner variable (transforming them in float by the same occasion). It does the same for the output coordinates putting them in a dim variable (also transforming them in float).
The getPerspectiveTransform and warpPerspective functions from Open CV are called.
The way the warping works is that it takes input and output coordinates and warp the given image so that the input coordinate A is now situated at the output coordinate A (therefor, in our code it puts the different corners at the point (0,0), (1000,0), (0,1000), (1000,1000); 1000 being the value for size[0] and size[1])
Finally, the function return the warped image to the main.

```python
def change_perpective(image, coords, size):
	transform_coords = np.zeros((4,2))
    
    #Goes through all the detected markers to assignated corners coordinate as input coordinates
	for el in coords:
		if el.get('ID') == 0:
			transform_coords[0] = el.get('POS')[2]
		elif el.get('ID') == 1:
			transform_coords[1] = el.get('POS')[3]
		elif el.get('ID') == 2:
			transform_coords[2] = el.get('POS')[1]
		elif el.get('ID') == 3:
			transform_coords[3] = el.get('POS')[0]
		else:
			pass
    
    #Declare corners (input coordinates) and dim (output coordinates)
	corners = np.float32(transform_coords)
	dim = np.float32([[0,0],[size[0],0],[0,size[1]],[size[0],size[1]]])

    #Warp the image
	perspective = cv2.getPerspectiveTransform(corners, dim)
	image = cv2.warpPerspective(image, perspective, (size[0], size[1]))
	
	return image

```

```python
def calculate_centroid(coords):
    x_center = 0
    y_center = 0
    for i in range(4):
        x_center += coords[i][0]/4
        y_center += coords[i][1]/4
    return [x_center, y_center]
```

In order to detect the different obstacles once all of the previously stated elements are detected, a threshold is applied to the image. This threshold allows to better identify the global obstacles, the environment being white and the obstacles being black (the function is givenbelow)

```python
def image_threshold(image, border_size, robot_coords, goal_coords):
    
    #Convert to grayscale
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #Reduce image noise
	blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    #Threshold image to better detect obstacle
	_, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
	
    #Remove robot and goal Aruco marker from contour detections
	thresh = delete_aruco(thresh, robot_coords)
	thresh = delete_aruco(thresh, goal_coords)

	kernel = np.ones((5, 5), np.uint8) 
	thresh = cv2.dilate(thresh, kernel, iterations=2)

    #Detect remaining obstacles
	contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
	cv2.drawContours(thresh, contours, -1, 0, border_size)
	
	return thresh
```

As we can see above, the image is first converted from BRG to grayscale in order to accuentuate the contrast between the elements. A gaussian blur is then applied, his goal being to reduce the noise of the image to improve the obstacle detection. Finally, a threshold is applied to the image. We first only used a binary threshold but the resulted algorithm was very dependent on the light conditions and we therefor added an Otsu threshold which allowed to reduce the impact lightning had on the detection.

The obstacle detection uses contour detection checking for any polygon it can find on the image. However, the Aruco markers used to represent the robot as well as the goal as also polygons, so, the delete_aruco function (given below) is called, in order to ignore those two.

```python
def delete_aruco(thresh, pos):
	max_p = [0, 0]
	min_p = [float('inf'), float('inf')]
	for point in pos:
		if point[0] > max_p[0]:
			max_p[0] = int(point[0])
		if point[0] < min_p[0]:
			min_p[0] = int(point[0])
		if point[1] > max_p[1]:
			max_p[1] = int(point[1])
		if point[1] < min_p[1]:
			min_p[1] = int(point[1])
	thresh = cv2.rectangle(thresh, min_p, max_p, 255, -1)
	return thresh
```

Then, we use the findContours and drawContours functions from Open CV to detect the global obstacles.

The function returning the thresholded image containing only the obstacles.

Finally, using the thresholded image containing the obstacles, a grid is created using the define_grid function (given below). This grid being used by the pathfinding algorithm.

```python
def define_grid(size, spacing, thresh):
	background = np.zeros([size[0], size[1], 1], dtype=np.uint8)

	vert = size[0]//spacing
	horz = size[1]//spacing
	v_blank = (size[0] % spacing)//2 + spacing
	h_blank = (size[1] % spacing)//2 + spacing

	coord_init = (v_blank, h_blank)
	grid = []
	coord = []

	for h in range(horz-1):
		coord_init = (h_blank + h*spacing, v_blank)
		for v in range(vert-1):
			if thresh.T[coord_init] == 255:
				background = cv2.circle(background, coord_init, 5, 255, -1)
				grid.append((h,v))
				coord.append(coord_init)
			coord_init = (coord_init[0], coord_init[1] + spacing)
			
	return grid, coord, background
```

It first created a zero filled matrix of the size of the image. An arbitrary spacing is creating so that the different point of the grid are separated by the same distance. A double for loop then goes through the entire matrix and check the value of the thresholded image obtained before at the position we are verifying. If the value is 255 (white), a white dot is put at the position meaning it is free of any obstacles and the coordinates are put in a list containing all the obstacle free coordinate.

The function then returns the grid, the completed background as well as the coordinates where there are no obstacles.

```python
def find_pos(pos, grid, coord):
	grid_c = None
	dist = float('inf')
	for point in coord:
		temp = euclidean_distance(point, pos)
		if temp < dist:
			grid_c = grid[coord.index(point)]
			dist = temp

	return grid_c
```

Finally, the function find_pos (given above) is called in the main, returning the corresponding position of the start and the goal on the grid we just created.

## 3. Path Finding

The goal of this section is to find the shortest path between two points in a grid-like structure while minimizing the total cost associated with the path. <br /><br />
We chose to use the A* Algorithm because it it garanteed to find the shortest path and it is more efficient than other algorithms in terms of both time and memory usage.<br/><br />
We use this algorithm to find the shortest path from the robot initial position to the robot targeted position. At this point of the project we are taking into account only the global obstacles visible from the camera, and not the local ones.


<div style="text-align: center;">
  <img src="images\a-star-path-example.jpg"alt="path" id="img_3.1" width="700" >
  <p style="text-align: center;">Image 3.1: Example of the shortest path from the Thymio robot (top left) to the target (bottom right) using A* Algorithm.</p>
</div>



The Node class initializes each node with its coordinates (coord) and the goal node's coordinates (goal).
- The g value is the cost from the start node to the current node, initialized to infinity.
- The _h value is the heuristic estimate  of the cost to reach the goal from the current node.
- The _f value is the sum of g and _h, used to determine the node's priority in the pathfinding process.
- The set_cost and set_parent methods are used to update a node's cost and its parent in the path.
- The _get_dist method calculates the heuristic distance (Manhattan distance) between two nodes, considering both straight and diagonal movements.

The A_star class is initialized with a grid and a set of camera coordinates.
- We chose 8 possible movements directions from a node.
- The _path_constructor method reconstructs the path from the start to the goal node, traversing backward from the goal.
- The _is_traversible method checks if a coordinate is within the grid and thus accessible.
- The find_path method implements the core A* algorithm. It uses a priority queue (heapq) to efficiently select the next node to explore. The method returns the constructed path from start to goal if available, or an empty list if no path is found.

## 4. Global Navigation

The goal of this section is to make the robot follow the desired path with the help of a controller. We compare two type of conroller :
- A *PD controller* 
- An *Astolfi controller*


### 4.1 PD coontroler

**proportional-derivative (PD) controller** is a type of feedback control system commonly used i control theory. It has two main components: *proportional (P) control* and *derivative (D) control*. 


The equation for a PD controller :  
$$c(t) = K_c(e(t) + T_d \frac{de}{dt}) + C$$


Therefore we defined a class **PD_controller** that contains : 
-  Initalisation : P (constant of proportionality of error signal), D (constant of proportionality of derivative of the error signal), time_step.
-  A control method that take the error and the previous error to calculate the controller output. 

### 4.2 Astolfi controller

To smoothly modulate the forward and rotational velocities, we can use an **Astolfi controller**.


<div style="text-align: center;">
  <img src="images\coord_astolfi.png"alt="Alt Text" id="img_3.2.1" width="300">
  <p style="text-align: center;">Image 4.2.1: Polar and global coordinates of the Thymio</p>
</div>



As shown in the figure above (<a href="#img_4.2.1">Image 3.2.1</a>), we can define the global coordinates of the robots by using his local coordinates by using those equations : 
<div style="text-align: center;">
  <img src="images\polar_astolfi.png" alt="Alt Text" id="img_4.2.2" width="300">
  <p style="text-align: center;">Image 4.2.2: Equations for polar coordinates</p>
</div>


The target is $\alpha,\rho,\beta$ to be equal to (0,0,0).

With this last set of equations, we can output the linear and angular velocities for the Thymio : 

<div style="text-align: center;">
  <img src="images\final_astolfi.png" alt="Alt Text" id="img_3.2.3" width="300">
  <p style="text-align: center;">Image 4.2.3: Output of the controler</p>
</div>


In our code this is all achived in the **astolfi_controller** class. We have :
- The initialisation method defined $k_{\rho}, k_{\alpha}, k_{\beta}$
- The **_calculate_parameter** method calculate $\rho,\alpha, \beta$ with the help of the goal and the postion of the robot.
- The **control** method use the **_calculate_parameter** method to return the linear and angular velocities $(v, \omega)$



### 4.3 Other functions for the navigation

Many functions were develloped to implement the navigation part. Here is a recap of the most important functions  : 







- The **calculate_centroid** function computes the centroid of the four corners of an aruco marker. The resulting centroid is returned as a 2-dimensional vector: $\text{{centroid}} = [(x_{\text{{center}}}),(y_{\text{{center}}})]$, where $x_{\text{{center}}} = \frac{x_1 + x_2 + x_3 + x_4}{4}$ and $y_{\text{{center}}} = \frac{y_1 + y_2 + y_3 + y_4}{4}$. The $x_{\text{{center}}}$ and $y_{\text{{center}}}$ are represented as an integer because we are working with pixels so we cannot use floating point numbers.


- The **calculate_centroid** function computes the centroid of the four corners of an aruco marker. The resulting centroid is returned as a 2-dimensional vector: $\text{{centroid}} = [(x_{\text{{center}}}),(y_{\text{{center}}})]$, where $x_{\text{{center}}} = \frac{x_1 + x_2 + x_3 + x_4}{4}$ and $y_{\text{{center}}} = \frac{y_1 + y_2 + y_3 + y_4}{4}$. The $x_{\text{{center}}}$ and $y_{\text{{center}}}$ are represented as an integer because we are working with pixels so we cannot use floating point numbers.


- The **calculate_error** function calculates the error between the robot's position and a given path. The input parameters include the path and the current coordinates of the robot. The function iterates through each segment of the path, determining the projection of the robot's position onto the path. If the robot moves beyond the current path segment, the function calculates the error based on the distance between the robot and the nearest endpoint of the path segment. The sign of the error indicates whether the robot is to the right or left of the path, as determined by the **rel_orientation** function. The resulting error is returned, considering the sign and the Euclidean distance between the robot's centroid and the projected position on the path.


## 6. Kalman Filter

The Extended Kalman Filter, an extension of the classic Kalman Filter for nonlinear systems, is an ideal tool for this scenario. It helps in predicting the future state of the robot by estimating its current position and movement, even in the absence of direct positional inputs from the external camera. 

### 6.1 Initial Approach

Initially, our approach to state estimation in the Thymio robot navigation project relied on using motor speed and camera inputs as the primary data sources for the Kalman filter. This setup seemed promising as it directly utilized the robot's inherent capabilities to infer its movement and orientation.


### 6.2 Motor Speed Noise

However, we soon encountered a significant challenge. The speed sensor in the Thymio robot exhibited a high degree of noise. This noise severely distorted the state estimation. For example, at a true speed of 50 mm/s, our estimates varied wildly between 20 and 80 mm/s. Such a wide margin of error was unacceptable for precise navigation and pose estimation.


### 6.3 State Space Equations for Differential Drive Robot

To address this, we shifted our focus to a more robust solution. We decided to predict the state of the robot based on the state space equations of a differential drive robot. This method relies less on noisy sensor data and more on the physical and theoretical model of the robot's movement.

In [ ]:
class extended_kalman:
	def __init__(self, Q, H, R, sampling_rate):
		self._Q = Q
		self._H = H
		self._R = R
		self._sampling_rate = sampling_rate        

	def predict(self, x_init, P_init, u):
		F = np.eye(3) #+ np.array([[0, 0, -u[0]*np.sin(x_init[2])], [0, 0, u[0]*np.cos(x_init[2])], [0, 0, 0]])*self._sampling_rate
		G = np.array([[np.cos(x_init[2]), 0], [np.sin(x_init[2]), 0], [0, -1]])*self._sampling_rate
		x_est = np.dot(F, x_init) + np.dot(G, u)
		P_est = np.dot(np.dot(F, P_init), F.T) + self._Q
		
		return x_est, P_est
	
	def correct(self, x_est, P_est, y):
		i = y - np.dot(self._H, x_est)
		j = np.eye(len(self._H))
		S = np.dot(self._H, np.dot(P_est, self._H.T)) + self._R
		Kn = np.dot(np.dot(P_est, self._H.T), np.linalg.inv(S))
		
		x_final = x_est + np.dot(Kn, i)
		P_final = np.dot(j, np.dot(P_est, j.T)) + np.dot(Kn, np.dot(self._R, Kn.T))
		
		return x_final, P_final
	

	def run(self, x_init, P_init, u, y):
		x_est, P_est = self.predict(x_init, P_init, u)
		return self.correct(x_est, P_est, y)

#### Initialization

The constructor __init__ initializes the filter with necessary parameters:

- `Q` : Process noise covariance matrix
- `H` : Measurement matrix
- `R` : Measurement noise covariance matrix
- `sampling_rate` : The rate at which the measurements are sampled

#### Prediction Step

The predict method implements the prediction step of the EKF.

- `x_init`: Initial state estimate
- `P_init` : Initial covariance estimate
- `u` : Control input vector
In this step, we calculate the estimated state x_est and the covariance of the estimate P_est using the state transition model and control input.

#### Correction Step

The correct method implements the correction step.

- `x_est`: Estimated state from the prediction step
- `P_est`: Estimated covariance from the prediction step
- `y` : Measurement vector

This step updates the estimated state and covariance based on the measurement y.

#### Running the Filter
The run method combines the predict and correct steps to process a single measurement.

`x_init`, `P_init`, `u`, `y`: Inputs for the prediction and correction steps